# Timing analysis

Here we evaluate the efficiency of the functions implemented in `src/prime_functions.py`. Of course, they weren't really intended to be optimal, but there are a few things it will be interesting to investigate - for example, how much of a difference does using the minimally-sized bases make in `miller_rabin`? 

First, we import some of the packages and functions that we'll need:

In [1]:
import timeit

import sys
sys.path.append("src/")
from prime_functions import get_primes, miller_rabin

## How much of a difference does numpy make?

There are two versions of the `get_primes` function, one which uses `numpy` and one which doesn't. We'd obviously expect the former to be faster, but how more efficient is it?

In [16]:
N = 1000000
print("With numpy:")
%timeit get_primes(N)
print("Without numpy:")
%timeit get_primes(N, with_numpy=False)

With numpy:
1.18 ms ± 932 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
Without numpy:
21.7 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


So that's a pretty decent speedup. Playing around, we see that there's some variation with the size of N and so on, but in general it seems that we can expect healthy gains by using the `numpy` version of the function. One natural question is, since it is sometimes necessary, how much of a difference does converting the `numpy` array to a list afterward make? Let's see:  

In [17]:
print("With numpy (and list conversion):")
%timeit list(get_primes(N))
print("Without numpy:")
%timeit get_primes(N, with_numpy=False)

With numpy (and list conversion):
3.37 ms ± 41.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Without numpy:
21.3 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Although there *is* an increase in runtime, the `numpy` version is still much faster. However, for smaller arrays, when the disparity is less pronounced, the additional time can be significant:

In [18]:
N = 1000
print("With numpy (and list conversion):")
%timeit list(get_primes(N))
print("Without numpy:")
%timeit get_primes(N, with_numpy=False)

With numpy (and list conversion):
24.6 µs ± 408 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Without numpy:
24.6 µs ± 44.3 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Despite this, overall it seems safe to conclude that there's little incentive to use the pure Python version, unless for some reason you absolutely cannot use `numpy`. 

So far we haven't considered how these implementations compare to a similar function from a standard maths library, so let's repeat the comparison with the `primerange` function from the `sympy` library, which computes all of the prime numbers within a certain range: 

In [4]:
from sympy import sieve

N = 100000000
print("With numpy:")
%timeit get_primes(N)
print("Without numpy:")
%timeit get_primes(N, with_numpy=False)
print("Sympy:")
%timeit list(sieve.primerange(1, N))

With numpy:
387 ms ± 5.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Without numpy:
3.05 s ± 56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Sympy:
829 ms ± 7.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


So the `numpy` version of `get_primes` is actually faster!

## How much better is it to use a smaller set of bases?

As for `get_primes`, there are effectively two versions of `miller_rabin`: one which uses the best known bases as far as possible, and one which builds the sets of bases from the first few prime numbers instead. Is there a notable difference in speed between the two? 

To test this, we use the following set of interesting primes and composites, which were also used for unit testing the `miller_rabin` function in `tests/test_prime_functions.py`:

In [5]:
primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47,
          53, 59, 61, 67, 71, 73, 79, 83, 89, 97, # All primes less than 100
          877, 27644437, 35742549198872617291353508656626642567, 359334085968622831041960188598043661065388726959079837, # Bell primes
          211, 2311, 200560490131, # Euclid 
          719, 5039, 39916801, 479001599, 87178291199, 10888869450418352160768000001, 265252859812191058636308479999999, 
          263130836933693530167218012159999999, 8683317618811886495518194401279999999, # Factorial 
          257, 65537, # Fermat 
          233, 1597, 28657, 514229, 433494437, 2971215073, 99194853094755497, 
          1066340417491710595814572169, 19134702400093278081449423917, # Fibonacci 
          113, 167, 269, 389, 419, 509, 659, 839, 1049, 1259, 1889, # Highly cototient 
          593, 32993, 2097593, 8589935681, 59604644783353249, 523347633027360537213687137, 
          43143988327398957279342419750374600193, # Leyland 
          127, 8191, 131071, 524287, 2147483647, 2305843009213693951, 618970019642690137449562111, 
          162259276829213363391578010288127, 170141183460469231731687303715884105727, # Mersenne 
          1361, 2521008887, 16022236204009818131831320183, # Mills 
          239, 9369319, 63018038201, 489133282872437279, 19175002942688032928599, # Newman-Shanks-Williams 
          40487, 6692367337, # Non-generous 
          5741, 33461, 44560482149, 1746860020068409, 68480406462161287469, 13558774610046711780701, 
          4125636888562548868221559797461449,  # Pell 
          211, 2309, 2311, 30029, 200560490131, 304250263527209, 23768741896345550770650537601358309, # Primorial 
          1111111111111111111, 11111111111111111111111, # Repunit 
          683, 2731, 43691, 174763, 2796203, 715827883, 2932031007403, 768614336404564651, 
          201487636602438195784363, 845100400152152934331135470251, 56713727820156410577229101238628035243, # Wagstaff
          383, 32212254719, 2833419889721787128217599, 195845982777569926302400511, 4776913109852041418248056622882488319 # Woodall
          ]

composites = [23**2, 233 * 239, # Simple composites 
              561, 41041, 825265, 321197185, 5394826801, 232250619601, 9746347772161, 1436697831295441,
              60977817398996785, 7156857700403137441, 1791562810662585767521, 87674969936234821377601,
              6553130926752006031481761, 1590231231043178376951698401, 35237869211718889547310642241,
              32809426840359564991177172754241, 2810864562635368426005268142616001, 
              349407515342287435050603204719587201, # Carmichael numbers
              25326001, 161304001, 960946321, 1157839381, 3215031751, 3697278427, 5764643587, 
              6770862367, 14386156093, 15579919981, 18459366157, 19887974881, 21276028621, # Strong pseudoprimes for bases 2, 3, 5
              3825123056546413051 # Strong pseudoprime for bases 2, 3, 5, 7, 11, 13, 17, 19, and 23 
              ] 

test_set = primes + composites

At this point we should note that the two versions of `miller_rabin` are identical for N greater than 2**64 = 18446744073709551616, so we should probably ignore any numbers in the test set that are larger than that:

In [7]:
print(f"Original size of test set: {len(test_set)}")
test_set = [*filter(lambda n : (n <= 18446744073709551616), test_set)]
print(f"Final size of test set: {len(test_set)}")

Original size of test set: 155
Final size of test set: 121


Okay, now let's time the two versions of the function for this test set: 

In [8]:
from functools import partial
without_minimal = partial(miller_rabin, minimal=False)

print("Without minimal bases:")
%timeit map(without_minimal, test_set)
print("With minimal bases:")
%timeit map(miller_rabin, test_set)

Without minimal bases:
108 ns ± 0.665 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
With minimal bases:
104 ns ± 0.989 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Although there is a small speedup, it's really not that impressive! 

Part of the problem may be the small size of the test set, as well as the fact that its entries vary quite widely in size. How about if we just consider, say, the numbers between 100000 and 300000, since the minimal version uses only one base in that case, whereas the alternative uses two?   

In [10]:
print("Without minimal bases:")
%timeit map(without_minimal, range(100000, 300001))
print("With minimal bases:")
%timeit map(miller_rabin, range(100000, 300001))

Without minimal bases:
272 ns ± 1.01 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
With minimal bases:
269 ns ± 1.54 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Again, only a very minor speedup! 

The ideal scenario for the minimal version is a range of numbers for which its set of bases is much smaller. So if we consider, for example, the sequence of one million consecutive integers starting at 2152302898747 then `miller_rabin` with `minimal=True` uses [2, 141889084524735, 1199124725622454117, 11096072698276303650] - i.e., 4 bases - whereas the alternative version uses [2, 3, 5, 7, 11, 13] - i.e., 6 bases. We would hope for good performance gains with the minimal basis, however we see that is not the case:

In [16]:
start, end = 2152302898747, 2152302898747 + 1000000
print("Without minimal bases:")
%timeit map(without_minimal, range(start, end))
print("With minimal bases:")
%timeit map(miller_rabin, range(start, end))

Without minimal bases:
285 ns ± 1.11 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
With minimal bases:
285 ns ± 1.06 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


This is interesting, albeit unwanted! It's unclear why we don't see any performance benefit - I'm not going to investigate too thoroughly here, but may take another look in the future...